This script was tested in google colab. Before starting, please cd to the folder where the script is. If missing module error happens, please install the corresponding library in this runtime. Please also don't forget to enable GPU in Runtime/Change runtime type

In [2]:
#cd to the script folder
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/483w_Capstone//Team B/code-proj1/data/3') #Need to change the address to where your script is
os.listdir('.')

Mounted at /content/drive


['data', 'log.csv', 'model.hdf5']

In [21]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds


AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 2

@tf.function
def scale(image):
  image = tf.image.convert_image_dtype(image, tf.float32)
  return image

@tf.function
def random_crop(images):
  boxes = tf.random.uniform(shape=(len(images), 4))
  box_indices = tf.random.uniform(shape=(len(images),), minval=0, maxval=BATCH_SIZE, dtype=tf.int32)
  images = tf.image.crop_and_resize(images, boxes, box_indices, (512, 512))
  return images

@tf.function
def scale_resize_image (image) :
  image = tf.image.convert_image_dtype(image, tf.float32) # equivalent to dividing image pixels by 255
  image = tf.image.resize(image, ( 512 , 512 ) ) # Resizing the image to 512x512 dimention
  return image
 



In [26]:
#load training dataset
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from pathlib import Path

nameList = os.listdir('data/label')
trainSize = len(nameList)

imgs = np.ndarray((trainSize,512,512,1), dtype='float32')
labels = np.ndarray((trainSize,512,512,1), dtype='float32')

for i, name in enumerate(nameList):
  stem = Path(name).stem
  img = load_img('data/train/' + stem + '.PNG', grayscale = True)
  label = load_img('data/label/' + name, grayscale = True)
  img = img_to_array(img).astype('float32')
  label = img_to_array(label).astype('float32')
  img -= img.mean()
  img /= img.std()
  img = scale_resize_image(img)
  label = scale_resize_image(label)
  imgs[i] = img
  labels[i] = label
  if i % 100 == 0:
    print('Done: {0}/{1} images'.format(i, len(imgs)))


labels /= 255
labels[labels > 0.5] = 1
labels[labels <= 0.5] = 0
labels = np.reshape(1-labels,[-1,262144,1])

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


Done: 0/98 images


In [ ]:
#training
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, CSVLogger
from keras import backend as keras
from keras.initializers import *
import tensorflow as tf
Adam = tf.keras.optimizers.Adam



def get_unet():
		inputs = Input((512,512,1))
		conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
		print ("conv1 shape:",conv1.shape)
		conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
		print ("conv1 shape:",conv1.shape)
		pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
		print ("pool1 shape:",pool1.shape)

		conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
		print ("conv2 shape:",conv2.shape)
		conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
		print ("conv2 shape:",conv2.shape)
		pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
		print ("pool2 shape:",pool2.shape)

		conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
		print ("conv3 shape:",conv3.shape)
		conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
		print ("conv3 shape:",conv3.shape)
		pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
		print ("pool3 shape:",pool3.shape)

		conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
		conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
		drop4 = Dropout(0.5)(conv4)
		pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

		conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
		conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
		drop5 = Dropout(0.5)(conv5)

		up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
		merge6 = concatenate([drop4,up6],axis=3)
		conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
		conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

		up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
		merge7 = concatenate([conv3,up7], axis = 3)
		conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
		conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

		up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
		merge8 = concatenate([conv2,up8], axis = 3)
		conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
		conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

		up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
		merge9 = concatenate([conv1,up9], axis = 3)
		conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
		conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
		conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
		conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
		reshape = Reshape((262144,1))(conv10)
		model = Model(inputs = inputs, outputs = reshape)
		model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
		return model

model = get_unet()
model_checkpoint = ModelCheckpoint('model.hdf5', monitor='loss',verbose=1, save_best_only=True)
csv_logger = CSVLogger('log.csv', append=False, separator=',')
model.summary()
model.fit(imgs, labels, batch_size=BATCH_SIZE, epochs=15, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint,csv_logger])
print('Fitting model...')

conv1 shape: (None, 512, 512, 64)
conv1 shape: (None, 512, 512, 64)
pool1 shape: (None, 256, 256, 64)
conv2 shape: (None, 256, 256, 128)
conv2 shape: (None, 256, 256, 128)
pool2 shape: (None, 128, 128, 128)
conv3 shape: (None, 128, 128, 256)
conv3 shape: (None, 128, 128, 256)
pool3 shape: (None, 64, 64, 256)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_24 (Conv2D)             (None, 512, 512, 64  640         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_25 (Conv2D)             (None, 512, 512, 64  36928       ['conv2d_24[0][0]']              
                                )                                                           

In [ ]:
#prediction
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, CSVLogger
from keras import backend as keras
from keras.initializers import *
import tensorflow as tf
Adam = tf.keras.optimizers.Adam


nameListTest = os.listdir('data/test')
testSize = len(nameListTest)
tests = np.ndarray((testSize,512,512,1), dtype='float32')
print('Loading')
for i,name in enumerate(nameListTest):
  img = load_img('data/test/' + name,grayscale = True)
  img = img_to_array(img).astype('float32')
  img -= img.mean()
  img /= img.std()
  tests[i] = img
  if (i+1) % 100 == 0:
    print('Done: {0}/{1} images'.format(i+1, testSize))
model = load_model('model.hdf5')
print("Network Loaded")
print('Predicting test data')
predictions = model.predict(tests, batch_size=1, verbose=1)
predictions = np.reshape(predictions, [-1,512,512,1])
for i in range(predictions.shape[0]):
		img = array_to_img(predictions[i])
		img.save("data/prediction/{:d}".format(i)+'.tif')

Loading


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


Done: 100/100 images
Network Loaded
Predicting test data
100/100 [==============================] - 7s 66ms/step
